<a href="https://colab.research.google.com/github/wheemin-2/25-1-ESAA/blob/main/0430_HW_Recommendations_Practice.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **콘텐츠 기반 필터링 실습 - TMDB 5000 영화 데이터 세트**

**장르 속성을 이용한 영화 콘텐츠 기반 필터링**

콘텐츠 기반 필터링이란, 사용자가 특정 영화를 감상하고 그 영화를 좋아했다면 그 영화와 비슷한 특성/속성/구성 요소 등을 가진 다른 영화를 추천하는 것

영화 간의 유사성을 판단하는 기준이 영화를 구성하는 다양한 콘텐츠(장르, 감독, 배우, 평점, 키워드, 영화 설명)가 됨

이 예제에서는 영화 장르 속성을 기반으로 콘텐츠 기반 필터링 추천 시스템을 구현

## **데이터 로딩 및 가공**

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import numpy as np
import warnings; warnings.filterwarnings('ignore')

movies = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/ESAA/25-1 OB/tmdb_5000_movies.csv')
print(movies.shape)
movies.head(1)

(4803, 20)


,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2009-12-10,2787965087,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800


데이터 설명
- 4803개의 레코드와 20개의 피처로 구성
- 영화 제목, 개요. 인 기도, 평점, 투표수, 예산, 키워드 등 영화에 대한 다양한 메타 정보를 가짐

추후 컨텐츠 기반 필터링 추천 분석에 사용할 주요 칼럼 설명
- id
- title, genres
- vote_average(평균 평점), vote_count(평점 투표수)
- popularity(영화의 인기)
- keywords(영화를 설명하는 주요 키워드)
- overview(영화에 대한 개요 설명)

In [ ]:
# 필요한 주요 칼럼만 추출
movies_df = movies[['id','title','genres','vote_average','vote_count','popularity','keywords','overview']]

In [ ]:
# genres, keywords : 파이썬 리스트 내부에 여러 딕셔너리가 묶여있는 형태의 문자열값을 가짐
pd.set_option('max_colwidth',100)
movies_df[['genres','keywords']][:1]

,genres,keywords
0,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""name"": ""Fantasy""}, {...","[{""id"": 1463, ""name"": ""culture clash""}, {""id"": 2964, ""name"": ""future""}, {""id"": 3386, ""name"": ""sp..."


In [ ]:
# 칼럼 내 문자열을 분해 : literal_eval() 함수
# 함수 적용 후 : 여러 딕셔너리로 구성된 객체를 가짐
from ast import literal_eval
movies_df['genres'] = movies_df['genres'].apply(literal_eval)
movies_df['keywords'] = movies_df['keywords'].apply(literal_eval)

In [ ]:
# 딕셔너리에서 'name' 키에 해당하는 값만 추출
movies_df['genres'] = movies_df['genres'].apply(lambda x : [y['name'] for y in x])
movies_df['keywords'] = movies_df['keywords'].apply(lambda x : [y['name'] for y in x])
movies_df[['genres','keywords']][:1]

,genres,keywords
0,"[Action, Adventure, Fantasy, Science Fiction]","[culture clash, future, space war, space colony, society, space travel, futuristic, romance, spa..."


## **장르 콘텐츠 유사도 측정**

*장르별 유사도 측정 방법*
1. genres를 문자열로 변경
2. CountVectorizer로 피처 벡터화
3. 변환된 행렬 데이터 값을 코사인 유사도로 비교

위의 과정을 통해 레코드별 타 레코드와 장르에서 코사인 유사도 값을 계산할 수 있음!

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

# CountVectorizer를 적용하기 위해 공백으로 word 단위가 구분되는 문자열로 변환
# join() 메서드 : 리스트 내의 각 요소를 하나의 문자열로 결합함
movies_df['genres_literal'] = movies_df['genres'].apply(lambda x : (' ').join(x))
count_vect = CountVectorizer(min_df=0.0, ngram_range=(1,2))
genre_mat = count_vect.fit_transform(movies_df['genres_literal'])
print(genre_mat.shape)

(4803, 276)


In [ ]:
# 코사인 유사도 계산
from sklearn.metrics.pairwise import cosine_similarity

genre_sim = cosine_similarity(genre_mat, genre_mat)
print(genre_sim.shape)
print(genre_sim[:2])

(4803, 4803)
[[1.         0.59628479 0.4472136  ... 0.         0.         0.        ]
 [0.59628479 1.         0.4        ... 0.         0.         0.        ]]


In [ ]:
# 레코드별로 유사도값이 높은 레코드의 위치값(인덱스) 저장
genre_sim_sorted_ind = genre_sim.argsort()[:,::-1]
print(genre_sim_sorted_ind[:1])

[[   0   46 3494 ... 3331 3333 2031]]


참고 : `np.argsort`를 2차원 행렬에 적용하면, 행방향 정렬과 열방향 정렬 두 방향이 존재함

`np.argsort(matrix, axis=1)` 에서 axis 설정을 통해 방향 선택 가능
- axis=1 : 행방향, default
- axis=0 : 열방향

## **장르 콘텐츠 필터링을 이용한 영화 추천**

장르 유사도에 따라 영화를 추천하는 `find_sim_movie()` 함수 생성

입력값 설명
- 기반 데이터 (movies_df)
- 레코드별 장르 코사인 유사도 인덱스
- 고객이 선정한 추천 기준이 되는 영화 제목
- 추천할 영화 건수



In [ ]:
def find_sim_movie(df, sorted_ind, title_name, top_n=10):
    # 인자로 입력된 movies_df 에서 'title' 칼럼이 입력된 title_name 값인 DF 추출
    title_movie = df[df['title']==title_name]

    # title_name을 가진 DF의 index 객체를 ndarray로 반환하고
    # sorted_ind 인자로 입력된 genre_sim_sorted_ind 객체에서 유사도 순으로 top_n개의 index 추출
    title_index = title_movie.index.values
    similar_indexes = sorted_ind[title_index, :(top_n)]

    # 추출된 top_n index 출력, top_n index는 2차원 데이터임
    # DF 에서 index로 사용하기 위해서 1차원 array로 변경
    print(similar_indexes)
    similar_indexes = similar_indexes.reshape(-1)

    return df.iloc[similar_indexes]

In [ ]:
# 영화 '대부'와 장르별로 유사한 영화 추천
similar_movies = find_sim_movie(movies_df, genre_sim_sorted_ind, 'The Godfather', 10)
similar_movies[['title','vote_average']]

[[1881 3378 3866 1370 1464  588 3887 3594 2839  892]]


,title,vote_average
1881,The Shawshank Redemption,8.5
3378,Auto Focus,6.1
3866,City of God,8.1
1370,21,6.5
1464,Black Water Transit,0.0
588,Wall Street: Money Never Sleeps,5.8
3887,Trainspotting,7.8
3594,Spring Breakers,5.0
2839,Rounders,6.9
892,Casino,7.8


문제점

- '대부'와 비슷하다고 하기 어려운 영화들이 리스트에 존재
- 평점이 낮은 경우들도 많음

*⭐좀 더 많은 후보군을 선정한 후 평점에 따라 필터링하여 최종 추천하는 방식으로 변경⭐*

In [ ]:
movies_df[['title','vote_average','vote_count']].sort_values('vote_average',ascending=False)[:10]

,title,vote_average,vote_count
4662,Little Big Top,10.0,1
3519,Stiff Upper Lips,10.0,1
4045,"Dancer, Texas Pop. 81",10.0,1
4247,Me You and Five Bucks,10.0,2
3992,Sardaarji,9.5,2
2386,One Man's Hero,9.3,2
1881,The Shawshank Redemption,8.5,8205
2970,There Goes My Baby,8.5,2
3337,The Godfather,8.4,5893
2796,The Prisoner of Zenda,8.4,11


평점의 문제점
- 소수의 관객이 특정 영화에 높은 평점을 부여한 경우, 다수의 관객에 대한 평점보다 높게 나올 수 있음 > 왜곡된 해석의 가능성!
- 평점에 평가 횟수를 반영할 수 있는 새로운 평가 방식이 필요함
    - 유명한 영화 평점 사이트인 IMDB에서는 평가 횟수에 대한 가중치가 부여된 평점(Weighted Rating) 방식을 사용


*가중평점 공식*

![weighted rating](https://img1.daumcdn.net/thumb/R1280x0/?scode=mtistory2&fname=https%3A%2F%2Fblog.kakaocdn.net%2Fdn%2Fbkhes6%2FbtrWllO80Ec%2FkLfwczHF9Nshq870X8HXxk%2Fimg.png)

 - v : 개별 영화에 평점을 투표한 횟수 (= 'vote_count')
 - m : 평점을 부여하기 위한 최소 투표 횟수
    - 투표 횟수에 따른 가중치를 직접 조절하는 역할
    - m ↑ : 평점 투표 횟수가 많은 영화에 더 많은 가중 평점을 부여
 - R : 개별 영화에 대한 평균 평점 (= 'vote_average')
 - C : 전체 영화에 대한 평균 평점 (= 'vote_average'의 평균)

In [ ]:
C = movies_df['vote_average'].mean()
# m : 전체 투표 횟수에서 상위 40%에 해당하는 횟수를 기준으로 결정
m = movies_df['vote_count'].quantile(0.6)
print('C:', round(C,3), 'm:', round(m,3))

C: 6.092 m: 370.2


In [ ]:
percentile=0.6
m = movies_df['vote_count'].quantile(percentile)
C = movies_df['vote_average'].mean()

def weighted_vote_average(record):
    v = record['vote_count']
    R = record['vote_average']

    return ( (v/(v+m)) * R + (m/(m+v)) * C )

movies_df['weighted_vote'] = movies_df.apply(weighted_vote_average, axis=1)

In [ ]:
# 가중평점 기준으로 상위 10개 영화 추출
movies_df[['title','vote_average','weighted_vote','vote_count']].sort_values(by='weighted_vote', ascending=False)[:10]

,title,vote_average,weighted_vote,vote_count
1881,The Shawshank Redemption,8.5,8.396052,8205
3337,The Godfather,8.4,8.263591,5893
662,Fight Club,8.3,8.216455,9413
3232,Pulp Fiction,8.3,8.207102,8428
65,The Dark Knight,8.2,8.136930,12002
1818,Schindler's List,8.3,8.126069,4329
3865,Whiplash,8.3,8.123248,4254
809,Forrest Gump,8.2,8.105954,7927
2294,Spirited Away,8.3,8.105867,3840
2731,The Godfather: Part II,8.3,8.079586,3338


`find_sim_movie()` 함수 수정
- 장르 유사성이 높은 영화를 top_n의 2배수만큼 후보군으로 선정
- weighted_vote 칼럼값이 높은 순으로 top_n만큼 추출

In [ ]:
def find_sim_movie(df, sorted_ind, title_name, top_n=10):
    title_movie = df[df['title']==title_name]
    title_index = title_movie.index.values

    # top_n의 2배에 해당하는 장르 유사성이 높은 인덱스 추출
    similar_indexes = sorted_ind[title_index, :(top_n*2)]
    similar_indexes = similar_indexes.reshape(-1)
    # 기준 영화 인덱스 제외
    similar_indexes = similar_indexes[similar_indexes != title_index]

    # top_n의 2배에 해당하는 후보군에서 weigthed_vote가 높은 순으로 top_n만큼 추출
    return df.iloc[similar_indexes].sort_values(by='weighted_vote', ascending=False)[:top_n]

similar_movies = find_sim_movie(movies_df, genre_sim_sorted_ind, 'The Godfather', 10)
similar_movies[['title','vote_average','weighted_vote']]

,title,vote_average,weighted_vote
1881,The Shawshank Redemption,8.5,8.396052
2731,The Godfather: Part II,8.3,8.079586
1847,GoodFellas,8.2,7.976937
3866,City of God,8.1,7.759693
1663,Once Upon a Time in America,8.2,7.657811
3887,Trainspotting,7.8,7.591009
883,Catch Me If You Can,7.7,7.557097
892,Casino,7.8,7.423040
4041,This Is England,7.4,6.739664
1149,American Hustle,6.8,6.717525


해당 추천시스템을 발전시킬 수 있는 방향
- 출연 배우, 감독까지 반영하여 추천
- 영화의 분위기, 개인이 좋아하는 성향을 반영하여 추천

# **아이템 기반 최근접 이웃 협업 필터링 실습**

최근접 이웃 협업 필터링은 사용자 기반과 아이템 기반으로 분류함. 이 중 일반적으로 추천 정확도가 더 뛰어난 것은 아이템 기반의 협업 필터링

In [2]:
import pandas as pd
import numpy as np

movies = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/ESAA/25-1 OB/movies.csv')
ratings = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/ESAA/25-1 OB/ratings.csv')
print(movies.shape)
print(ratings.shape)

(9742, 3)
(100836, 4)


- movies : 영화에 대한 메타 정보인 title과 genres를 가지는 영화 정보
- ratings : 사용자별로 영화에 대한 평점을 매긴 데이터 세트
    - rating : 최소 0.5, 최대 5, 0.5 단위로 부여됨

## **데이터 가공 및 변환**

In [3]:
display(movies.head(3))
display(ratings.head(3))

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance


,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224


row level 형태의 원본 데이터 세트를 알맞은 형태로 변환해야함
- 모든 개별 사용자를 row, 모든 영화를 column으로 구성한 데이터 세트로 변경
- `pivot_table()` 함수 : row level의 값을 칼럼으로 변경하는 데에 효과적

![df1](https://img1.daumcdn.net/thumb/R1280x0/?scode=mtistory2&fname=https%3A%2F%2Fblog.kakaocdn.net%2Fdn%2FbYxrFR%2FbtrTtFohm7l%2FIcdAaKAkRG0hI13wNJOXwK%2Fimg.png)

In [4]:
ratings = ratings[['userId','movieId','rating']]
ratings_matrix = ratings.pivot_table('rating',index='userId',columns='movieId')
ratings_matrix.head(3)

movieId,1,2,3,4,5,6,7,8,9,10,...,193565,193567,193571,193573,193579,193581,193583,193585,193587,193609
userId,,,,,,,,,,,,,,,,,,,,,
1,4.0,NaN,4.0,NaN,NaN,4.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
# title 칼럼을 얻기 위해 movies와 조인
rating_movies = pd.merge(ratings, movies, on='movieId')

# title 칼럼으로 pivot 수행
ratings_matrix = rating_movies.pivot_table('rating', index='userId', columns='title')

# NaN 값을 모두 0으로 채우기
ratings_matrix = ratings_matrix.fillna(0)
ratings_matrix.head(3)

title,'71 (2014),'Hellboy': The Seeds of Creation (2004),'Round Midnight (1986),'Salem's Lot (2004),'Til There Was You (1997),'Tis the Season for Love (2015),"'burbs, The (1989)",'night Mother (1986),(500) Days of Summer (2009),*batteries not included (1987),...,Zulu (2013),[REC] (2007),[REC]² (2009),[REC]³ 3 Génesis (2012),anohana: The Flower We Saw That Day - The Movie (2013),eXistenZ (1999),xXx (2002),xXx: State of the Union (2005),¡Three Amigos! (1986),À nous la liberté (Freedom for Us) (1931)
userId,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## **영화 간 유사도 산출**

`cosine_similarity()` 함수를 이용해 유사도 산출
- `cosine_similarity()` 함수는 **행을 기준으로 서로 다른 행을 비교**하여 유사도를 산출
- 현재 ratings_matrix는 userId가 기준인 행 레벨 데이터이므로 이 데이터에 cosine_similarity()를 적용하면 영화 간의 유사도가 아닌 사용자 간의 유사도가 만들어지게 됨!!
    - ratings_matrix의 행과 열의 위치를 변경하는 과정이 필요
    - 행과 열을 변경하여 행 기준이 영화가 되도록 설정 > cosine_simliarity()는 영화 간의 유사도를 산출하게 됨

In [6]:
ratings_matrix_T = ratings_matrix.transpose()
ratings_matrix_T.head(3)

userId,1,2,3,4,5,6,7,8,9,10,...,601,602,603,604,605,606,607,608,609,610
title,,,,,,,,,,,,,,,,,,,,,
'71 (2014),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0
'Hellboy': The Seeds of Creation (2004),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
'Round Midnight (1986),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [7]:
from sklearn.metrics.pairwise import cosine_similarity

item_sim = cosine_similarity(ratings_matrix_T, ratings_matrix_T)

# cosine_similarity()로 반환된 넘파이 행렬에 영화명을 매핑해 DF로 반환
item_sim_df = pd.DataFrame(data=item_sim, index=ratings_matrix.columns, columns=ratings_matrix.columns)

print(item_sim_df.shape)
item_sim_df.head(3)

(9719, 9719)


title,'71 (2014),'Hellboy': The Seeds of Creation (2004),'Round Midnight (1986),'Salem's Lot (2004),'Til There Was You (1997),'Tis the Season for Love (2015),"'burbs, The (1989)",'night Mother (1986),(500) Days of Summer (2009),*batteries not included (1987),...,Zulu (2013),[REC] (2007),[REC]² (2009),[REC]³ 3 Génesis (2012),anohana: The Flower We Saw That Day - The Movie (2013),eXistenZ (1999),xXx (2002),xXx: State of the Union (2005),¡Three Amigos! (1986),À nous la liberté (Freedom for Us) (1931)
title,,,,,,,,,,,,,,,,,,,,,
'71 (2014),1.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.141653,0.0,...,0.0,0.342055,0.543305,0.707107,0.0,0.0,0.139431,0.327327,0.0,0.0
'Hellboy': The Seeds of Creation (2004),0.0,1.000000,0.707107,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,...,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0
'Round Midnight (1986),0.0,0.707107,1.000000,0.0,0.0,0.0,0.176777,0.0,0.000000,0.0,...,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0


In [8]:
# 영화 '대부'와 유사도가 높은 상위 6개 영화 추출
item_sim_df['Godfather, The (1972)'].sort_values(ascending=False)[:6]

,"Godfather, The (1972)"
title,
"Godfather, The (1972)",1.000000
"Godfather: Part II, The (1974)",0.821773
Goodfellas (1990),0.664841
One Flew Over the Cuckoo's Nest (1975),0.620536
Star Wars: Episode IV - A New Hope (1977),0.595317
Fargo (1996),0.588614


In [9]:
# '인셉션'과 유사도가 높은 상위 5개 영화 추출 (인셉션 제외)
item_sim_df['Inception (2010)'].sort_values(ascending=False)[1:6]

,Inception (2010)
title,
"Dark Knight, The (2008)",0.727263
Inglourious Basterds (2009),0.646103
Shutter Island (2010),0.617736
"Dark Knight Rises, The (2012)",0.617504
Fight Club (1999),0.615417


## **아이템 기반 최근접 이웃 협업 필터링으로 개인화된 영화 추천**

개인화된 영화 추천의 가장 큰 특징
- 개인이 아직 관람하지 않은 영화를 추천한다는 것!
- 아직 관람하지 않은 영화에 대해서 아이템 유사도와 기존에 관람한 영화의 평점 데이터를 바탕으로 새롭게 모든 영화의 예측 평점을 계산한 후, 높은 예측 평점을 가진 영화를 추천하는 방식

![pred_rating](https://velog.velcdn.com/images%2Fgjtang%2Fpost%2Fb2c544ec-6333-4a07-86a8-f261cfa27960%2Fimage.png)

$\sum_N (\left| S_{i,N} \right|)$ 은 정규화를 위해 나누는 값


In [10]:
def predict_rating(ratings_arr, item_sim_arr):
    ratings_pred = ratings_arr.dot(item_sim_arr)/np.array([np.abs(item_sim_arr).sum(axis=1)])
    return ratings_pred

In [11]:
ratings_pred = predict_rating(ratings_matrix.values, item_sim_df.values)
ratings_pred_matrix = pd.DataFrame(data=ratings_pred, index=ratings_matrix.index,
                                   columns=ratings_matrix.columns)
ratings_pred_matrix.head(3)

title,'71 (2014),'Hellboy': The Seeds of Creation (2004),'Round Midnight (1986),'Salem's Lot (2004),'Til There Was You (1997),'Tis the Season for Love (2015),"'burbs, The (1989)",'night Mother (1986),(500) Days of Summer (2009),*batteries not included (1987),...,Zulu (2013),[REC] (2007),[REC]² (2009),[REC]³ 3 Génesis (2012),anohana: The Flower We Saw That Day - The Movie (2013),eXistenZ (1999),xXx (2002),xXx: State of the Union (2005),¡Three Amigos! (1986),À nous la liberté (Freedom for Us) (1931)
userId,,,,,,,,,,,,,,,,,,,,,
1,0.070345,0.577855,0.321696,0.227055,0.206958,0.194615,0.249883,0.102542,0.157084,0.178197,...,0.113608,0.181738,0.133962,0.128574,0.006179,0.212070,0.192921,0.136024,0.292955,0.720347
2,0.018260,0.042744,0.018861,0.000000,0.000000,0.035995,0.013413,0.002314,0.032213,0.014863,...,0.015640,0.020855,0.020119,0.015745,0.049983,0.014876,0.021616,0.024528,0.017563,0.000000
3,0.011884,0.030279,0.064437,0.003762,0.003749,0.002722,0.014625,0.002085,0.005666,0.006272,...,0.006923,0.011665,0.011800,0.012225,0.000000,0.008194,0.007017,0.009229,0.010420,0.084501


예측 평가 지표로 MSE를 사용하여 성능 확인
- 이때 사용자가 영화의 평점을 주지 않은 경우에도 (0으로 기록) 예측 점수가 부여되는 경우가 많이 발생한다는 점을 유의해야함
- 따라서 실제와 예측평점의 차이는 기존에 평점이 부여되었던 데이터에 대해서만(즉, true 값이 존재하는 데이터) 오차 정도를 측정하는 것이 합리적
- 계산 결과 실제 값과 예측값은 서로 스케일이 다르기 때문에 MSE 값이 크게 나올 수 있음. 중요한 것은 MSE를 감소시키는 방향으로 개선하는 것 !

In [12]:
from sklearn.metrics import mean_squared_error

# 사용자가 평점을 부여한 영화에 대해서만 예측 성능 평가 MSE를 구함
def get_mse(pred, actual):
    # 평점이 있는 실제 영화만 추출
    pred = pred[actual.nonzero()].flatten()
    actual = actual[actual.nonzero()].flatten()
    return mean_squared_error(pred, actual)

print('아이템 기반 모든 최근접 이웃 MSE:', get_mse(ratings_pred, ratings_matrix.values))

아이템 기반 모든 최근접 이웃 MSE: 9.895354759094706


특정 영화와 가장 비슷한 유사도를 가지는 영화에 대해서만 유사도 벡터를 적용하는 함수 생성
- 아쉬운 점 : 수행시간이 오래 걸림 (행, 열 별로 for 문 반복 수행)

In [13]:
def predict_rating_topsim(ratings_arr, item_sim_arr, n=20):
    # 사용자-아이템 평점 행렬 크기만큼 0으로 채운 예측 행렬 초기화
    pred = np.zeros(ratings_arr.shape)

    # 사용자-아이템 평점 행렬의 열 크기만큼 루프 수행
    for col in range(ratings_arr.shape[1]):
        # 유사도 행렬에서 유사도가 큰 순으로 n개 데이터 행렬의 인덱스 반환
        top_n_items = [np.argsort(item_sim_arr[:, col])[:-n-1:-1]]
        # 개인화된 예측 평점을 계산
        for row in range(ratings_arr.shape[0]):
            pred[row, col] = item_sim_arr[col, :][top_n_items].dot(ratings_arr[row,:][top_n_items].T)
            pred[row, col] /= np.sum(np.abs(item_sim_arr[col,:][top_n_items]))

    return pred

In [14]:
# 예측 평점 계산 후 실제 평점과의 MSE 구하기
ratings_pred = predict_rating_topsim(ratings_matrix.values, item_sim_df.values, n=20)
print('아이템 기반 최근접 Top-20 이웃 MSE:', get_mse(ratings_pred, ratings_matrix.values))

# 계산된 예측 평점 데이터는 DataFrame으로 재생성
ratings_pred_matrix = pd.DataFrame(data=ratings_pred, index=ratings_matrix.index,
                                   columns=ratings_matrix.columns)

<ipython-input-13-6dbc8d16cb22>:14: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  pred[row, col] = item_sim_arr[col, :][top_n_items].dot(ratings_arr[row,:][top_n_items].T)


아이템 기반 최근접 Top-20 이웃 MSE: 3.694409449382562


특정 사용자에 대해 영화를 추천하는 알고리즘 구현

In [15]:
# userId = 9인 사용자에 대해 영화 추천하기
# 사용자가 좋아하는 영화 확인
user_rating_id = ratings_matrix.loc[9,:]
user_rating_id[user_rating_id > 0].sort_values(ascending=False)[:10]

,9
title,
Adaptation (2002),5.0
Austin Powers in Goldmember (2002),5.0
Back to the Future (1985),5.0
Citizen Kane (1941),5.0
"Lord of the Rings: The Fellowship of the Ring, The (2001)",5.0
"Lord of the Rings: The Two Towers, The (2002)",5.0
"Producers, The (1968)",5.0
Raiders of the Lost Ark (Indiana Jones and the Raiders of the Lost Ark) (1981),5.0
Elling (2001),4.0


In [17]:
# 사용자가 평점을 주지 않은 영화를 리스트로 반환하는 함수 생성
def get_unseen_movies(ratings_matrix, userId):
    user_rating = ratings_matrix.loc[userId, :]
    # 사용자가 이미 관람한 영화 리스트
    already_seen = user_rating[user_rating > 0].index.tolist()
    # 모든 영화명을 리스트 객체로 생성
    movies_list = ratings_matrix.columns.tolist()
    # list comprehension으로 아직 관람하지 않은 영화 리스트 생성
    unseen_list = [ movie for movie in movies_list if movie not in already_seen]

    return unseen_list

In [18]:
def recomm_movie_by_user_id(pred_df, userId, unseen_list, top_n=10):
    # 예측 평점 DataFrame에서 사용자id와 unseen list로 들어온 영화명 칼럼을 추출해
    # 가장 예측 평점이 높은 순으로 정렬
    recomm_movies = pred_df.loc[userId, unseen_list].sort_values(ascending=False)[:top_n]
    return recomm_movies

# 사용자가 관람하지 않은 영화명 추출
unseen_list = get_unseen_movies(ratings_matrix, 9)

# 아이템 기반의 최근접 이웃 협업 필터링으로 영화 추천
recomm_movies = recomm_movie_by_user_id(ratings_pred_matrix, 9, unseen_list, top_n=10)

# 평점 데이터를 DF로 생성
recomm_movies = pd.DataFrame(data=recomm_movies.values, index=recomm_movies.index,
                             columns=['pred_score'])

recomm_movies


,pred_score
title,
Shrek (2001),0.866202
Spider-Man (2002),0.857854
"Last Samurai, The (2003)",0.817473
Indiana Jones and the Temple of Doom (1984),0.816626
"Matrix Reloaded, The (2003)",0.800990
Harry Potter and the Sorcerer's Stone (a.k.a. Harry Potter and the Philosopher's Stone) (2001),0.765159
Gladiator (2000),0.740956
"Matrix, The (1999)",0.732693
Pirates of the Caribbean: The Curse of the Black Pearl (2003),0.689591
